# Resynchronization des traces - (Jitter 1)

Nous savons que le side-channel est efficace pour attaquer une implémentation cryptographique. 

------------

**Aims**
- Observer la trace d'execution de l'algorithms

**Plan**
1. Import packages
2. Loading & plotting data
3. Trying CPA reverse on traces
5. Trying CPA on synchronized traces


<img src='images/DoIt.png' style='width: 100px'>

# import

In [ ]:
import warnings
import estraces
import numpy as np
import matplotlib.pyplot as plt
import time
import scared
from scipy.signal import find_peaks
import tqdm

warnings.filterwarnings('ignore')
plt.rcParams["figure.figsize"] = [20, 5]

# Collect traces from the Chipwhisperer (optional)

**If you have a ChipWhisperer**, you can collect the traces needed for this lab. If **you do not have the necessary equipment**, proceed to the next title `Loading & plotting datas`.

## Setup your chipwhisperer
---
To collect traces with Chipwhisperer (cw), you'll need to select a `SCOPETYPE`, a `PLATFORM`, and a `CRYPTO_TARGET`.
* `SCOPETYPE` is `'OPENADC'` for the CWLite
* `PLATFORM` is the target device, with `'CWLITEARM'`
* `CRYPTO_TARGET` selects the crypto implementation, with `'TINYAES128C'` working on all platforms.

```python
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_1_1'
```

In [ ]:
# Define the values for the following constants
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_1_1'

### 🫵 Your turn: Setup your cw by using `Setup_Scripts/Setup_Generic.ipynb`

In [ ]:
%run "../chipwhisperer/chipwhisperer-jupyter/Setup_Scripts/Setup_Generic.ipynb"

### 🫵 Your turn: Using the cross-compiler, compile the vulnerable AES code for your victim processor. Don't forget the option `EXTRA_OPTS=ADD_JITTER`

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../chipwhisperer/firmware/mcu/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 EXTRA_OPTS=ADD_JITTER SS_VER=$3

### 🫵 Your turn: Load the vulnerable AES with jitter 1 onto the victim processor.

In [ ]:
cw.program_target(scope, prog, "../chipwhisperer/firmware/mcu/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

---
Your chipwhisperer is ready !
## Run an AES computation and collect the power trace
### 🫵 Your turn: Execute the AES on your victim processor

💻[cw documentation](https://chipwhisperer.readthedocs.io/en/latest/scope-api.html)

> 💡**Tips**: Follow these steps to run AES and launch the scope.
> - Generate key, plain
> - set_key on target
> - write the plain onto the target with simpleserial_write and 'p' parameter
> - Define the number of samples to collect into the trace
> - Arm the scope for trace acquisition
> - Run the AES
> - Get the result (ciphertext) from the target with simpleserial_read and 'r' parameter
> - store the trace collected frome the scope with get_last_trace

In [ ]:
ktp = cw.ktp.Basic()

In [ ]:
# Generate plaintext and key for the AES execution
key, plaintext = ktp.next()

# Set the AES key for the target
target.set_key(key)

# Send the plaintext to encrypt
target.simpleserial_write('p', plaintext)

# Define the number of samples to collect into the trace
scope.adc.samples = 5_000

# Arm the scope for trace acquisition
scope.arm()

# Run the AES
ret = scope.capture()
if ret:
    print("Target timed out!")

#Store the result
ciphertext_returned = np.array(target.simpleserial_read('r', 16))

#Store the trace collected
trace = np.array(scope.get_last_trace())

### 🫵 Your turn: Print the ciphertext

In [ ]:
ciphertext_returned

### 🫵 Your turn: Plot the trace captured

In [ ]:
plt.rcParams['figure.figsize']=(16,4)
plt.title('Trace AES collectée')
plt.plot(trace)
plt.show()

### 🫵 Your turn: Using the previous cell to collect 2'000 traces

In [ ]:
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []
keyin_array = []
response_array = []

key, text = ktp.next()

target.set_key(key)

scope.adc.samples = 8_000
#scope.adc.samples = 24_000


# Number of traces
#N = 2500
N = 100

# Loop for traces collection 
for i in tnrange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    keyin_array.append(key)
    response_array.append(response)
    
    key, text = ktp.next() 

# Loading & plotting datas

**If you have a ChipWhisperer and you have collect the traces skip only this cell.** 
### 🫵 Your turn: Import the ets file `TP_1_CWLite_AES_traces_set.ets`

In [ ]:
ths = estraces.read_ths_from_ets_file('../SideSCA-Traces-Public/TP_1_CWLite_AES_traces_set.ets')

### 🫵 Your turn: Display information contains in ets file

In [ ]:
print(ths)

### 🫵 Your turn: Plot a trace contains in the ets file

💻[matplotlib documentation](https://numpy.org/devdocs/user/absolute_beginners.html)

❓Questions
- Do you recognize AES in the trace?

In [ ]:
# create an array with the ths for each element (sample, ciphertext, plaintext, key)
traces_np_array = ths.samples
ciphertext_np_array = ths.ciphertext
plaintext_np_array = ths.plaintext
key_np_array = ths.key

In [ ]:
# verify that the shapes of the arrays are the same (only for cipher, plain and key)
print(ciphertext_np_array.shape)
print(plaintext_np_array.shape)
print(key_np_array.shape)

In [ ]:
#%matplotlib
traces_np_array = ths.samples
plt.rcParams['figure.figsize']=(16,4)
plt.title('Trace AES collectée')
plt.plot(traces_np_array[0])
plt.show()

### 🫵 Your turn: Plot 10 traces and their first 1'000 points

You can see that the traces are out of sync with each other. This is due to the addition of jitter.

In [ ]:
plt.rcParams['figure.figsize']=(18,5)
for i in range(10):
    plt.plot(traces_np_array[i][0:1000])
plt.show()

# Trying CPA reverse on traces

### 🤓☝️ Remember
The `CPA reverse` allows to determine if an operation in an AES in a round, in this case, is vulnerable to a CPA attack.

The desynchronization of the traces may have blocked the CPA attack. We must try the reverse CPA to see if it is possible to attack the traces directly.

### 🫵 Your turn: Do the reverse on the output of SB in the first round:

💻[scared documentation](https://eshard.gitlab.io/scared/api_reference/analysis/reverse.html)

In [ ]:
# Write the selection function with scared to attack SB in first round
def aes_function_firstRde_SBout(plaintext, key):
    return scared.aes.encrypt(plaintext = plaintext, key = key, at_round=1, after_step=scared.aes.base.Steps.SUB_BYTES)
    
Selection_function = scared.reverse_selection_function(aes_function_firstRde_SBout)

In [ ]:
# Place the traces in the container
container = scared.Container(ths)

In [ ]:
# Prepare the reverse attack with scared
Reverse_SB = scared.CPAReverse(selection_function = Selection_function, model = scared.HammingWeight())


In [ ]:
#run the attack
Reverse_SB.run(container)

In [ ]:
#display the dimension of result array
Reverse_SB.results.shape

### 🫵 Your turn : Plot the result of the CPA reverse

❓ Questions
- Is there a leak? 


In [ ]:
plt.rcParams['figure.figsize']=(16,4)
plt.plot(Reverse_SB.results.T)
plt.show()